# ALINE Migraine Simulator - Validation & QA

**Ticket 004**: Bayesian / Markov Simulator for Synthetic Migraine Episodes

This notebook validates the synthetic migraine data generated by the simulator.

## Objectives:
1. Load and explore the synthetic datasets
2. Visualize latent state dynamics
3. Analyze migraine probability distributions
4. Examine temporal patterns and correlations
5. Validate realistic behavior

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from pathlib import Path

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ Libraries imported successfully")

## 1. Load Data and Metadata

In [ ]:
# Load metadata
with open('../data/synthetic_migraine_meta.json', 'r') as f:
    metadata = json.load(f)

print("📊 Dataset Metadata:")
print(f"  Generation date: {metadata['generation_date']}")
print(f"  Total users: {metadata['config']['n_users']}")
print(f"  Horizon: {metadata['config']['horizon']} days")
print(f"  Train split: {metadata['config']['train_split']}")
print()
print("Train Stats:")
print(f"  Records: {metadata['dataset_stats']['train']['n_records']:,}")
print(f"  Users: {metadata['dataset_stats']['train']['n_users']:,}")
print(f"  Migraine rate: {metadata['dataset_stats']['train']['migraine_rate']:.3f}")
print()
print("Validation Stats:")
print(f"  Records: {metadata['dataset_stats']['val']['n_records']:,}")
print(f"  Users: {metadata['dataset_stats']['val']['n_users']:,}")
print(f"  Migraine rate: {metadata['dataset_stats']['val']['migraine_rate']:.3f}")

In [ ]:
# Load a sample of training data (first 10,000 records for visualization)
print("Loading sample training data...")
train_sample = pd.read_csv('../data/synthetic_migraine_train.csv', nrows=10000)

print(f"\n📈 Data shape: {train_sample.shape}")
print(f"Columns: {list(train_sample.columns[:10])}...")
print(f"\nFirst few rows:")
train_sample.head()

## 2. Latent State Dynamics

In [ ]:
# Visualize latent states for a single user over time
user_id = 0
user_data = train_sample[train_sample['user_id'] == user_id]

fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle(f'Latent State Evolution - User {user_id}', fontsize=16, fontweight='bold')

latent_states = ['Z_stress', 'Z_sleepDebt', 'Z_hormonal', 'Z_envLoad']
colors = ['red', 'blue', 'purple', 'green']

for idx, (state, color) in enumerate(zip(latent_states, colors)):
    ax = axes[idx // 2, idx % 2]
    ax.plot(user_data['day'], user_data[state], color=color, linewidth=1.5, alpha=0.7)
    ax.set_xlabel('Day')
    ax.set_ylabel('Latent State Value')
    ax.set_title(state.replace('Z_', '').capitalize())
    ax.grid(True, alpha=0.3)
    
    # Highlight migraine days
    migraine_days = user_data[user_data['migraine'] == 1]['day']
    for day in migraine_days:
        ax.axvline(x=day, color='orange', alpha=0.3, linewidth=0.5)

plt.tight_layout()
plt.show()

print(f"🟠 Orange lines indicate migraine days for User {user_id}")

In [ ]:
# Distribution of latent states across all users
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Latent State Distributions (All Users)', fontsize=16, fontweight='bold')

for idx, (state, color) in enumerate(zip(latent_states, colors)):
    ax = axes[idx // 2, idx % 2]
    ax.hist(train_sample[state], bins=50, color=color, alpha=0.6, edgecolor='black')
    ax.set_xlabel('Latent State Value')
    ax.set_ylabel('Frequency')
    ax.set_title(f'{state.replace("Z_", "").capitalize()} Distribution')
    ax.axvline(train_sample[state].mean(), color='darkred', linestyle='--', 
               linewidth=2, label=f'Mean: {train_sample[state].mean():.2f}')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3. Migraine Probability & Occurrence

In [ ]:
# Migraine probability distribution
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Probability distribution
axes[0].hist(train_sample['migraine_prob'], bins=50, color='purple', alpha=0.6, edgecolor='black')
axes[0].set_xlabel('Migraine Probability')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Migraine Probabilities')
axes[0].axvline(train_sample['migraine_prob'].mean(), color='red', linestyle='--', 
               linewidth=2, label=f'Mean: {train_sample["migraine_prob"].mean():.3f}')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Migraine occurrence vs probability
migraine_yes = train_sample[train_sample['migraine'] == 1]['migraine_prob']
migraine_no = train_sample[train_sample['migraine'] == 0]['migraine_prob']

axes[1].hist([migraine_no, migraine_yes], bins=30, color=['green', 'red'], 
            alpha=0.6, label=['No Migraine', 'Migraine'], edgecolor='black')
axes[1].set_xlabel('Migraine Probability')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Probability Distribution by Migraine Outcome')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Migraine rate: {train_sample['migraine'].mean():.3f}")
print(f"Average probability: {train_sample['migraine_prob'].mean():.3f}")

In [ ]:
# Migraine rate by user
user_migraine_rates = train_sample.groupby('user_id')['migraine'].mean()

plt.figure(figsize=(12, 5))
plt.hist(user_migraine_rates, bins=30, color='coral', alpha=0.7, edgecolor='black')
plt.xlabel('Migraine Rate (per user)')
plt.ylabel('Number of Users')
plt.title('Distribution of Migraine Rates Across Users')
plt.axvline(user_migraine_rates.mean(), color='red', linestyle='--', 
           linewidth=2, label=f'Mean: {user_migraine_rates.mean():.3f}')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"User-level migraine rate statistics:")
print(user_migraine_rates.describe())

## 4. Feature Correlations with Migraines

In [ ]:
# Correlation between latent states and migraine occurrence
correlations = {}
for state in latent_states:
    correlations[state.replace('Z_', '')] = train_sample[state].corr(train_sample['migraine'])

plt.figure(figsize=(10, 5))
plt.bar(correlations.keys(), correlations.values(), color=['red', 'blue', 'purple', 'green'], alpha=0.7)
plt.xlabel('Latent State')
plt.ylabel('Correlation with Migraine')
plt.title('Latent State Correlations with Migraine Occurrence')
plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

print("Correlations:")
for k, v in correlations.items():
    print(f"  {k}: {v:.4f}")

In [ ]:
# Compare latent states on migraine vs non-migraine days
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Latent States: Migraine vs Non-Migraine Days', fontsize=16, fontweight='bold')

for idx, (state, color) in enumerate(zip(latent_states, colors)):
    ax = axes[idx // 2, idx % 2]
    
    migraine_vals = train_sample[train_sample['migraine'] == 1][state]
    no_migraine_vals = train_sample[train_sample['migraine'] == 0][state]
    
    ax.hist([no_migraine_vals, migraine_vals], bins=40, color=['lightblue', 'red'], 
            alpha=0.6, label=['No Migraine', 'Migraine'], edgecolor='black')
    ax.set_xlabel('Latent State Value')
    ax.set_ylabel('Frequency')
    ax.set_title(state.replace('Z_', '').capitalize())
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # Add mean lines
    ax.axvline(migraine_vals.mean(), color='darkred', linestyle='--', linewidth=2)
    ax.axvline(no_migraine_vals.mean(), color='darkblue', linestyle='--', linewidth=2)

plt.tight_layout()
plt.show()

## 5. Temporal Patterns

In [ ]:
# Migraine rate over time (averaged across users)
daily_migraine_rate = train_sample.groupby('day')['migraine'].mean()

plt.figure(figsize=(15, 5))
plt.plot(daily_migraine_rate.index, daily_migraine_rate.values, color='darkred', linewidth=1.5, alpha=0.7)
plt.axhline(train_sample['migraine'].mean(), color='blue', linestyle='--', 
           linewidth=2, label=f'Overall Mean: {train_sample["migraine"].mean():.3f}')
plt.xlabel('Day')
plt.ylabel('Migraine Rate')
plt.title('Daily Migraine Rate (Averaged Across Users)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Temporal variation in migraine rate: std = {daily_migraine_rate.std():.4f}")

## 6. Validation Summary

In [ ]:
print("=" * 70)
print("✅ VALIDATION SUMMARY - TICKET 004")
print("=" * 70)
print()
print("📊 Dataset Quality:")
print(f"  ✓ Migraine rate: {train_sample['migraine'].mean():.3f} (~15% target)")
print(f"  ✓ Users simulated: {train_sample['user_id'].nunique()}")
print(f"  ✓ Days per user: {train_sample.groupby('user_id')['day'].count().mean():.0f}")
print()
print("🧠 Latent State Characteristics:")
for state in latent_states:
    mean = train_sample[state].mean()
    std = train_sample[state].std()
    print(f"  ✓ {state}: mean={mean:.2f}, std={std:.2f}")
print()
print("📈 Migraine Patterns:")
print(f"  ✓ Probability range: [{train_sample['migraine_prob'].min():.3f}, {train_sample['migraine_prob'].max():.3f}]")
print(f"  ✓ Average probability: {train_sample['migraine_prob'].mean():.3f}")
print(f"  ✓ User-level variation: std={user_migraine_rates.std():.3f}")
print()
print("🔍 Feature Integration:")
feature_cols = [col for col in train_sample.columns if col not in 
               ['user_id', 'day', 'Z_stress', 'Z_sleepDebt', 'Z_hormonal', 'Z_envLoad', 
                'migraine_prob', 'migraine']]
print(f"  ✓ Features included: {len(feature_cols)}")
print(f"  ✓ Sample features: {feature_cols[:5]}")
print()
print("=" * 70)
print("✅ Simulator validated successfully!")
print("   Ready for ALINE model training (Ticket 005)")
print("=" * 70)